Considere o modelo de p-mediana

- $I = \{1,\dots, m\}$: o conjunto de localidades disponíveis
- $J = \{1, \dots, n\}$: o conjunto de clientes para as localidades

- $p$: número de localidades a serem abertas
- $d_j$: o tamanho da demanda do cliente $j \in J$
- $c_{ij}$: o custo de transporte entre a localidade $i \in I$ e o cliente $j \in J$

- $x_{ij}$: a fração de demanda do cliente $j \in J$ atendida pela localidade $i \in I$
- $y_i$: vale 1 se a localidade é aberta na locação $i \in I$, 0 caso contrário.

\begin{aligned}
    Z^*=\min & \sum_{i \in \mathcal{I}} \sum_{j \in \mathcal{J}} d_j c_{i j} x_{i j} \\
    \text { s.t. } & \sum_{i \in \mathcal{I}} x_{i j}=1 \quad \forall j \in \mathcal{J} \\
    & \sum_{i \in \mathcal{I}} y_i=p \\
    & x_{i j} \leq y_i \quad \forall i \in \mathcal{I}, j \in \mathcal{J} \\
    & y_i \in\{0,1\} \quad \forall i \in \mathcal{I} \\
    & x_{i j} \geq 0 \quad \forall i \in \mathcal{I}, j \in \mathcal{J}
\end{aligned}


Vamos dualizar o primeiro conjunto de restrições, multiplicando por $\{\lambda_j \geq 0: j \in N\}$ à cada restrição

Resultando no seguinte subproblema lagrangeano:

\begin{aligned}
    Z_D(\lambda)=\min & \sum_{i \in \mathcal{I}} \sum_{j \in \mathcal{J}} d_j c_{i j} x_{i j}+\sum_{j \in \mathcal{J}} \lambda_j\left(1-\sum_{i \in \mathcal{I}} x_{i j}\right) \\
    =\min & \sum_{i \in \mathcal{I}} \sum_{j \in \mathcal{J}}\left(d_j c_{i j}-\lambda_j\right) x_{i j}+\sum_{j \in \mathcal{J}} \lambda_j \\
    \text { s.t. } & \sum_{i \in \mathcal{I}} y_i=p \\
    & x_{i j} \leq y_i \quad \forall i \in \mathcal{I}, j \in \mathcal{J} \\
    & y_i \in\{0,1\} \quad \forall i \in \mathcal{I} \\
    & x_{i j} \geq 0 \quad \forall i \in \mathcal{I}, j \in \mathcal{J}
\end{aligned}

### Instância

In [1]:
d = [10 6 20 32 15 28 3 19 8 1]

1×10 Matrix{Int64}:
 10  6  20  32  15  28  3  19  8  1

In [2]:
c = [
10 6 20 32 15 28 3 19 8 13
10 7 11 12 32 15 20 26 4 41
13 17 31 37 21 5 13 15 14 12
4 13 14 22 8 31 26 11 12 23
21 21 13 18 9 27 11 16 26 32
32 18 11 14 11 11 16 32 34 8
15 9 13 12 14 15 32 8 12 9
28 32 15 2 17 12 9 6 11 6  
]

8×10 Matrix{Int64}:
 10   6  20  32  15  28   3  19   8  13
 10   7  11  12  32  15  20  26   4  41
 13  17  31  37  21   5  13  15  14  12
  4  13  14  22   8  31  26  11  12  23
 21  21  13  18   9  27  11  16  26  32
 32  18  11  14  11  11  16  32  34   8
 15   9  13  12  14  15  32   8  12   9
 28  32  15   2  17  12   9   6  11   6

In [3]:
p = 3

3

In [4]:
locations = 1:size(c,1) # the set, I
customers = 1:length(d) # the set, J

1:10

### Encontrando Lower Bounds

Este subproblema lagrangeano tem uma estrutura especial que faz com que seja mais fácil de resolver. Como relaxamos o primeiro conjunto de restrições, não precisamos nos importar em atribuir demanda à todos os clientes.

Dessa forma, basta seguirmos o seguinte algoritmo para resolver por inspeção:

1. Para cada $i \in I$, vamos computar $v_i = \sum_{j \in J} min \{0,  d_jc_{ij} - \lambda_j\}$
2. Ordenamos os candidatos a locação pelo valor de $v_i$ (do menor para o maior) e selecionaremos os $p$ $v_i's$ mais negativos.
3. Fazemos $y_i = 1$ para as $p$ localidades escolhidas
4. Fazemos $x_{ij} = 1$ se $d_jc_ij - \lambda_j < 0$

In [7]:
#Implementação de 1
lambda = zeros(size(customers))

v = Array{Float64}(undef, size(locations))
for i in locations
    v[i] = 0
    for j in customers
        v[i] = v[i] + min(0, d[j]*c[i,j] - lambda[j])
    end
end

In [11]:
#Implementação de 2
idx = sortperm(v)

8-element Vector{Int64}:
 1
 2
 3
 4
 5
 6
 7
 8

In [ ]:
#Implementação de 3
y = zeros(Int, size(locations))

for i in 1:p 
    y[idx[i]] = 1
end 

In [ ]:
#Implementação de 4
x = zeros(Int, length(locations), length(customers))
for i in locations, j in customers
    if y[i] == 1 && d[j]*c[i,j]-lambda[j]<0
        x[i,j] = 1
    end
end

Além disso, precisamos calcular o valor de $Z_D(\lambda)$

In [ ]:
Z_D = 0.0
for j in customers
    Z_D = Z_D + lambda[j]
    for i in locations
        Z_D = Z_D + d[j]*c[i,j]*x[i,j] - lambda[j]*x[i,j]
    end
end

Que é essencialmente isto:

$$
Z_D(\lambda)=\min \sum_{i \in \mathcal{I}} \sum_{j \in \mathcal{J}} d_j c_{i j} x_{i j}+\sum_{j \in \mathcal{J}} \lambda_j\left(1-\sum_{i \in \mathcal{I}} x_{i j}\right)
$$

Juntando tudo e botando em uma função, teremos:

In [17]:
function lowerBound(lambda)
    
    # Passo 1: Calculando v
    v = Array{Float64}(undef, size(locations))
    for i in locations
        v[i] = 0
        for j in customers
            v[i] = v[i] + min(0, d[j] * c[i, j] - lambda[j])
        end
    end

    # Passo 2: Ordenando v do mais negativo ao menos negativo, pegando os índices
    idx = sortperm(v)

    # Passo 3: Identificando y
    y = zeros(Int, size(locations))

    for i = 1:p
        y[idx[i]] = 1
    end

    # Passo 4: Identificando x
    x = zeros(Int, length(locations), length(customers))
    for i in locations, j in customers
        if y[i] == 1 && d[j] * c[i, j] - lambda[j] < 0
            x[i, j] = 1
        end
    end

    # Calculando o lower bound
    Z_D = 0.0
    for j in customers
        Z_D = Z_D + lambda[j]
        for i in locations
            Z_D = Z_D + d[j] * c[i, j] * x[i, j] - lambda[j] * x[i, j]
        end
    end

    return Z_D, x, y
end

lowerBound (generic function with 1 method)

### Encontrando upper bounds

Resolvendo o subproblema lagrangeano, nós temos uma solução explícita pra **x** e para **y**. No entanto, note que a variável y satisfaz todas as restrições originais, mas a variável x não, uma vez que dualizamos o conjunto de restrições de atendimento a toda demanda. Dessa forma, para encontrarmos **soluções viáveis** para o problema e ter um limite superior, vamos manter os valores de y encontrados e para cada cliente, vamos atribuir o que tiver mais próximo das facilidades abertas. 

In [16]:
function upperBound(y)
    # Calculando x, dado y a partir da proximidade
    x = zeros(Int, length(locations), length(customers))

    for j in customers
        fac_prox = 10000
        idx = -1
        for i in locations
            if y[i] == 1 && c[i, j] <= fac_prox
                idx = i
                fac_prox = c[i, j]
            end
        end
        x[idx, j] = 1
    end

    # Calculando Z
    Z = 0.0
    for i in locations
        for j in customers
            Z = Z + d[j] * c[i, j] * x[i, j]
        end
    end

    return Z, x
end

upperBound (generic function with 1 method)

### Atualizando o multiplicador de lagrange

Para atualizar o multiplicador $\lambda^k$ em cada iteração $k$, vamos utilizar o método do subgradiente:

$$
\lambda_j^{k+1}=\lambda_j^k+t_k\left(1-\sum_{i \in \mathcal{I}} x_{D i j}^k\right)
$$

onde o tamanho de passo $t_k$ é determinado por
$$
t_k=\frac{\theta_k\left(Z_{\mathrm{UB}}-Z_D\left(\lambda^k\right)\right)}{\sum_{j \in \mathcal{J}}\left(1-\sum_{i \in \mathcal{I}} x_{D i j}^k\right)^2}
$$

Tal que $Z_{UB}$ é o melhor limite superior encontrado (nesse caso, o menor), $x_{D i j}^k$ é o elemento em $(i,j)$ da solução $x_D^k$ do subproblema lagrangeano na iteração $k$, e $\theta_k \in (0,2]$ um escalar.

Para rodar iterações de um algoritmo lagrageano atualizando o multiplicador como descrito antes, vamos precisar de algumas premissas. Primeiro, vamos definir o número máximo de iterações.

In [ ]:
MAX_ITER = 10000

Também vamos preparar dois objetos do tipo Float64 vazios para armazenar os limites inferiores e superiores encontrados na iteração k

In [ ]:
UB = Array{Float64}(undef, 0)
LB = Array{Float64}(undef, 0)

Para armazenar os melhores limites encontrados durante todo o algoritmo, definimos:

In [ ]:
Z_UB = Inf
Z_LB = -Inf

Também armazenamos a melhor solução encontrada durante o algoritmo, isto é, os valores das variáveis da melhor solução viável encontrada

In [ ]:
x_best = zeros(length(locations), length(customers))
y_best = zeros(length(locations))

Por fim, preparamos o multiplicador de lagrange $\lambda$ inicializando com os valores zerados:

In [12]:
lambda = zeros(size(customers))

10-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

Agora podemos calcular o tamanho de passo para atualizar o multiplicador. Nesse exemplo, vamos fazer $\theta_k = 1$ por simplicidade:

In [ ]:
theta = 1.0

Agora nós calculamos o vetor onde o j-ésimo elemento é:
$$
1-\sum_{i \in \mathcal{I}} x_{D i j}^k
$$

In [ ]:
residual = 1 .- transpose(sum(x_D, dims = 1))

Atualizando o multiplicador:

In [ ]:
t = theta * (Z_UB - Z_D) / sum(residual .^ 2)
lambda = lambda + t * residual

De forma geral, a cara do algoritmo deve seguir algo assim:

In [ ]:
for k=1:MAX_ITER
  ...

  if opt_gap < 0.000001
    break
  end
end

Ou seja, repetiremos todos esses procemos até que o gap de otimalidade seja menor do que um $\epsilon$. O gap de otimalidade é dado pela diferença entre o melhor limite superior com o melhor limite inferior.

Dentro do loop, nós fazemos o seguinte:
- Obtemos um limite inferior resolvendo o subproblema lagrangeano
- Depois com as locations abertas do subproblema, buscamos uma solução viável

In [ ]:
Z_D, x_D, y = lowerBound(lambda, p)
Z, x = upperBound(y)

Depois disso, atualizamos os melhores limites encontrados e guardamos a melhor solução

In [ ]:
# Atualizando o upper bound
if Z < Z_UB
  Z_UB = Z
  x_best = x
  y_best = y
end

# Atualizando lower bound
if Z_D > Z_LB
  Z_LB = Z_D
end


Juntando tudo em uma única função, teremos:

In [13]:
function lagrangianRelaxation(p)
    # Número máximo de iterações permitidas no algoritmo
    MAX_ITER = 10000

    # Definindo vetores vazios do tipo Float64 para os limites superiores e inferiores
    UB = Array{Float64}(undef, 0) # Limite superior
    LB = Array{Float64}(undef, 0) # Limite inferior

    # Os melhores upper and lower bounds
    Z_UB = Inf
    Z_LB = -Inf

    # A melhor solução viável encontrada
    x_best = zeros(length(locations), length(customers))
    y_best = zeros(length(locations))

    # Inicializando multiplicadores de lagrange com zero do tamanho de clientes
    lambda = zeros(size(customers))

    for k = 1:MAX_ITER

        # Obtendo limites inferiores e superiores
        Z_D, x_D, y = lowerBound(lambda) # resolvendo o subproblema lagrangeano
        Z, x = upperBound(y) # buscando uma solução viável

        # Atualizando o upper bound
        if Z < Z_UB
            Z_UB = Z
            x_best = x
            y_best = y
        end

        # Atualizando o lower bound
        if Z_D > Z_LB
            Z_LB = Z_D
        end

        # Determinando o tamanho de passo e atualizando o multiplicador
        theta = 1.0
        residual = 1 .- transpose(sum(x_D, dims = 1))
        t = theta * (Z_UB - Z_D) / sum(residual .^ 2)
        lambda = lambda + t * residual

        # Calculando o gap de otimizalidade (1a condição de parada)
        opt_gap = (Z_UB - Z_LB) / Z_UB
        if opt_gap < 0.000001
            print("Saindo por otimalidade!")
            break
        end

    end

    return Z_UB, x_best, y_best, UB, LB
end

lagrangianRelaxation (generic function with 1 method)

### Solução

Primeiro, vamos resolver utilizando o solver:

In [14]:
using JuMP, Gurobi

function optimal(p)
    m = Model(Gurobi.Optimizer)

    @variable(m, x[i in locations, j in customers] >= 0)
    @variable(m, y[i in locations], Bin)

    @objective(m, Min, sum( d[j]*c[i,j]*x[i,j]
                     for i in locations, j in customers) )

    @constraint(m, [j in customers], sum( x[i,j] for i in locations) == 1)
    @constraint(m, sum( y[i] for i in locations) == p)
    @constraint(m, [i in locations, j in customers], x[i,j] <= y[i] )
    
    set_silent(m)
    JuMP.optimize!(m)

    Z_opt = JuMP.objective_value(m)
    x_opt = JuMP.value.(x)
    y_opt = JuMP.value.(y)

    return Z_opt, x_opt, y_opt
end

optimal (generic function with 1 method)

In [15]:
Z_opt, x_opt, y_opt = optimal(p)
println("Solução ótima = ", Z_opt)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-09
Solução ótima = 957.0


In [18]:
Z_UB, x_best, y_best, UB, LB = lagrangianRelaxation(p)
println("\nSolução ótima = ", Z_UB)

Saindo por otimalidade!
Solução ótima = 957.0


Outras condições de parada:

In [ ]:
# Começar com theta grande e ir diminuindo a medida que não há melhora nos melhores limites. Parar se theta for muito pequeno.
theta = 2
improve = 0
theta_min = 0.0001

...

if improve >= maxIter/20
    theta = theta/2
    improve = 0
    if theta < theta_min
        println("Parando por theta pequeno (iteração ", k, ")")
        break
    end
end

In [ ]:
# Se o tamanho de passo for muito pequeno
if t < theta_min
    break
end 